## 1. Importação

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from typing import List, Dict, Tuple

path_to_dir = '../../raw/Presencial2025_dados_raw/'

df_presencial_cursos = pd.read_csv(f'{path_to_dir}dados_presencial_cursos.csv')
df_presencial_disciplinas = pd.read_csv(f'{path_to_dir}dados_presencial_disciplinas.csv')

df_info_questionario = pd.read_csv(f'{path_to_dir}info_questionario_presencial.csv')
df_info_perguntas = pd.read_csv(f'{path_to_dir}info_perguntas_presencial.csv')
df_info_dimensoes = pd.read_csv(f'{path_to_dir}info_dimensoes_presencial.csv')

## 2. Entendendo cada DataFrame

In [2]:
def dist_variavel(df,column,savefig=True,path_to_dir='./temp_images_presencial/',title='teste'):
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=f'{column}', order=df[f'{column}'].value_counts().index)
    plt.xticks(rotation=45)
    plt.title(f'Distribuição de {column}')
    plt.ylabel('Contagem')
    plt.xlabel(f'{column}')
    plt.tight_layout()
    if savefig:
        plt.savefig(f'{path_to_dir}{title}')
    plt.close()

### 2.1 `df_presencial_cursos`

* `COD_CURSO`
* `CURSO` : Necessita de split (CURSO,PERIODO,LOCALIDADE)
* `SETOR_CURSO`

In [3]:
df_presencial_cursos['TEMP'] = df_presencial_cursos['CURSO'].str.split('-').apply(lambda x: [item.strip() for item in x])

In [4]:
df_presencial_cursos['MODALIDADE'] = df_presencial_cursos['TEMP'].str[1]
df_presencial_cursos['LOCAL'] = df_presencial_cursos['TEMP'].str[2]
df_presencial_cursos['CURSO'] = df_presencial_cursos['TEMP'].str[0]

#df_presencial_cursos.drop(columns=['TEMP'], inplace=True)
df_presencial_cursos.head()

,COD_CURSO,CURSO,SETOR_CURSO,TEMP,MODALIDADE,LOCAL
0,40001016027G0,ENGENHARIA AMBIENTAL E SANITÁRIA,CENTRO DE ESTUDOS DO MAR,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",Presencial,Pontal do Paraná
1,40001016047G0,FILOSOFIA,SETOR DE CIÊNCIAS HUMANAS,"[FILOSOFIA, Presencial, Curitiba]",Presencial,Curitiba
2,40001016030G0,ENGENHARIA CIVIL,SETOR DE TECNOLOGIA,"[ENGENHARIA CIVIL, Presencial, Curitiba]",Presencial,Curitiba
3,40001016005G0,ARQUITETURA E URBANISMO,SETOR DE TECNOLOGIA,"[ARQUITETURA E URBANISMO, Presencial, Curitiba]",Presencial,Curitiba
4,40001016076G0,MEDICINA VETERINÁRIA,SETOR DE CIÊNCIAS AGRÁRIAS,"[MEDICINA VETERINÁRIA, Presencial, Curitiba]",Presencial,Curitiba


In [5]:
columns = ['MODALIDADE','LOCAL']
for i in columns:
    dist_variavel(df_presencial_cursos,i,title=f'cursos/Distribuicao_de_{i}_before')

In [6]:
counts = df_presencial_cursos['MODALIDADE'].value_counts()
low_modalidades = counts[counts < 10].index
indices = df_presencial_cursos[df_presencial_cursos['MODALIDADE'].isin(low_modalidades)].index.tolist()
print('Indíces a serem arrumados: ',indices)

df_temp = df_presencial_cursos.loc[indices,:]

df_temp_expanded = pd.DataFrame(df_temp['TEMP'].tolist(), index=df_temp.index)
df_temp_expanded.columns = ['CURSO_RAW', 'MODALIDADE_RAW', 'LOCAL_RAW','ALGO']
display(df_temp_expanded)

Indíces a serem arrumados:  [37, 63, 67, 68, 71, 83, 95]


,CURSO_RAW,MODALIDADE_RAW,LOCAL_RAW,ALGO
37,EDUCAÇÃO ESPECIAL E INCLUSIVA,PARFOR,a Distância,Curitiba
63,PETRÓLEO E GÁS INTEGRADO AO ENSINO MÉDIO,Técnico,Presencial,Curitiba
67,GESTÃO CULTURAL,a Distância,Curitiba,None
68,EDUCAÇÃO BILÍNGUE DE SURDOS,PARFOR,Presencial,Curitiba
71,HISTÓRIA,MEMÓRIA E IMAGEM,Presencial,Curitiba
83,LETRAS,LIBRAS,Presencial,Curitiba
95,EDUCAÇÃO DO CAMPO,CIÊNCIAS DA NATUREZA,Presencial,Matinhos


In [7]:
def change_row(df,row,columns,values_change):
    for i,j in zip(columns,values_change):
        df.loc[row,i] = j

# ---- #

# 37 - PARFOR é um programa da UFPR que adequa profissionais para suas áreas de licenciatura através de cursos. Mais informações: https://prograp.ufpr.br/parfor/ | Solução: Arrumar os valores
change_row(df_presencial_cursos,37,['MODALIDADE','LOCAL'],['a Distância','Curitiba'])
if 37 in indices: 
    indices.remove(37) # 37 -> Modelo a Distância; remover

# ---- #

# 63 - Solução: Arrumar valores
change_row(df_presencial_cursos,63,['MODALIDADE','LOCAL'],['Presencial','Curitiba'])
if 63 in indices:
    indices.remove(63) # 63 -> Arrumado

# ---- #

# 67 - Solução: Remover
change_row(df_presencial_cursos,67,['MODALIDADE','LOCAL'],['a Distância','Curitiba'])
if 67 in indices: 
    indices.remove(67) # 67 -> Modelo a Distância; remover

# ---- #

# 68 - Solução: Arrumar valores
change_row(df_presencial_cursos,68,['MODALIDADE','LOCAL'],['Presencial','Curitiba'])
if 68 in indices:
    indices.remove(68) # 68 -> Arrumado

# ---- #

# 71 - Solução: Arrumar valores
change_row(df_presencial_cursos,71,['MODALIDADE','LOCAL'],['Presencial','Curitiba'])
if 71 in indices:
    indices.remove(71) # 71 -> Arrumado

# ---- #

# 83 - Solução: Arrumar valores
change_row(df_presencial_cursos,83,['MODALIDADE','LOCAL'],['Presencial','Curitiba'])
if 83 in indices:
    indices.remove(83) # 83 -> Arrumado

# ---- #

# 95 - Solução: Arrumar valores
change_row(df_presencial_cursos,95,['MODALIDADE','LOCAL'],['Presencial','Curitiba'])
if 95 in indices:
    indices.remove(95) # 95 -> Arrumado

# ---- #

df_temp = df_presencial_cursos.loc[indices,:]

df_temp_expanded = pd.DataFrame(df_temp['TEMP'].tolist(), index=df_temp.index)
if len(indices) != 0:
    display(df_temp_expanded)
else:
    columns = ['MODALIDADE','LOCAL']
    
    for i in columns:
        dist_variavel(df_presencial_cursos,i,title=f'cursos/Distribuicao_de_{i}_after')

Devemos manter `df_presencial_cursos.loc[100,:]`? Ele se refere ao PROGRAD

### 2.2 `df_presencial_disciplinas`

In [8]:
df_presencial_disciplinas['TEMP'] = df_presencial_disciplinas['CURSO'].str.split('-').apply(lambda x: [item.strip() for item in x])
df_presencial_disciplinas.head(2)

,COD_DISCIPLINA,NOME_DISCIPLINA,COD_CURSO,MULTIPLA_ESCOLHA,CURSO,IDPROGRAMA,SETOR_CURSO,TEMP
0,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,40001016027G0,CENTRO DE ESTUDOS DO MAR,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,..."
1,EAS172-EAD,QUALIDADE DA ÁGUA,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,40001016027G0,CENTRO DE ESTUDOS DO MAR,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,..."


In [ ]:


df_presencial_disciplinas['MODALIDADE'] = df_presencial_disciplinas['TEMP'].str[1]
df_presencial_disciplinas['LOCAL'] = df_presencial_disciplinas['TEMP'].str[2]
df_presencial_disciplinas['CURSO'] = df_presencial_disciplinas['TEMP'].str[0]

df_presencial_disciplinas

columns = ['MODALIDADE','LOCAL']
for i in columns:
    dist_variavel(df_presencial_disciplinas,i,title=f'disciplinas/Distribuicao_de_{i}_before')

In [ ]:
counts = df_presencial_disciplinas['MODALIDADE'].value_counts()
low_modalidades = counts[counts < 10].index
indices = df_presencial_disciplinas[df_presencial_disciplinas['MODALIDADE'].isin(low_modalidades)].index.tolist()
print('Indíces a serem arrumados: ',indices)

df_temp = df_presencial_disciplinas.loc[indices,:]

df_temp_expanded = pd.DataFrame(df_temp['TEMP'].tolist(), index=df_temp.index)
display(df_temp_expanded)

Indíces a serem arrumados:  [960, 961, 962, 963, 964]


,0,1,2
960,GESTÃO CULTURAL,a Distância,Curitiba
961,GESTÃO CULTURAL,a Distância,Curitiba
962,GESTÃO CULTURAL,a Distância,Curitiba
963,GESTÃO CULTURAL,a Distância,Curitiba
964,GESTÃO CULTURAL,a Distância,Curitiba


### 2.3 `df_info_questionario`

In [10]:
m

NameError: name 'm' is not defined

### 2.4 `df_info_perguntas`

### 2.5 `df_info_dimensoes`